<a href="https://colab.research.google.com/github/sgatheca/2027_Elections/blob/main/Voter_Projections_2027.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Projected 2027 Registered Voters and Turnout Rate**


This Python script uses the pandas library to process election data from an Excel file.  It projects the number of registered voters for 2027 using the Compound Annual Growth Rate (CAGR) and projects the voter turnout for 2027 using a weighted average of historical turnout data.  Finally, it saves the updated data, including the projections, to a new Excel file.
To get the cleaned Excel file data, I have extracted elections data from IEBC website (2013, 2017 and 2022). The data was in .pdf format.

In [1]:
#Import Libraries
import numpy as np
import pandas as pd

In [2]:
#Load data from excel file
df=pd.read_excel('Voters 2013_22.xlsx')

In [3]:
#show the first 5 columns
df.head(5)

,code,County,registered_Voters_2013,Turnout_2013,registered_Voters_2017,Turnout_2017,registered_Voters_2022,Turnout_2022
0,1,MOMBASA,408747,0.6662,580644,0.5896,641913,0.4317
1,2,KWALE,174443,0.7200,281102,0.6588,328253,0.5452
2,3,KILIFI,336132,0.6500,508425,0.6448,588602,0.4848
3,4,TANA RIVER,79454,0.8100,118338,0.7345,141096,0.6626
4,5,LAMU,52346,0.8400,69793,0.7051,81453,0.6146


In [4]:
#Remane columns to make them easier to work with
df.rename(
    columns={
        "code": "County Code",
        "County": "County Name",  # Added County Name
        "registered_Voters_2013": "Registered Voters 2013",
        "Turnout_2013": "Turnout 2013",
        "registered_Voters_2017": "Registered Voters 2017",
        "Turnout_2017": "Turnout 2017",
        "registered_Voters_2022": "Registered Voters 2022",
        "Turnout_2022": "Turnout 2022",
    },
    inplace=True,
)

In [5]:
df.head(2)

,County Code,County Name,Registered Voters 2013,Turnout 2013,Registered Voters 2017,Turnout 2017,Registered Voters 2022,Turnout 2022
0,1,MOMBASA,408747,0.6662,580644,0.5896,641913,0.4317
1,2,KWALE,174443,0.7200,281102,0.6588,328253,0.5452


In [18]:
# 1. Project Number of Registered Voters for 2027 using CAGR
# Function to calculate CAGR
def calculate_cagr(beginning_value, ending_value, num_years):
    if beginning_value == 0:
        return 0  # Return 0 to avoid division by zero
    return (ending_value / beginning_value) ** (1 / num_years) - 1


# Function to project future registered voters using CAGR
def project_registered_voters(voters_2013, voters_2017, voters_2022):
    # Use the voter data directly from the arguments
    if not all(isinstance(v, (int, np.integer)) for v in [voters_2013, voters_2017, voters_2022]):
        return None
    if voters_2013 is None or voters_2017 is None or voters_2022 is None:
        return None
    # Calculate CAGR between 2013 and 2022 (9 years)
    cagr_2013_2022 = calculate_cagr(voters_2013, voters_2022, 9)
    cagr_2017_2022 = calculate_cagr(voters_2017, voters_2022, 5)

    # Project registered voters for 2027 using CAGR
    projected_voters_2027_from_2022 = voters_2022 * (1 + cagr_2017_2022) ** 5
    return projected_voters_2027_from_2022


# Apply the projection to each row and store the results
df["Voters_2027"] = df.apply(
    lambda row: project_registered_voters(
        row["Registered Voters 2013"],
        row["Registered Voters 2017"],
        row["Registered Voters 2022"],
    ),
    axis=1,
)


In [19]:
df.head(2)

,County Code,County Name,Registered Voters 2013,Turnout 2013,Registered Voters 2017,Turnout 2017,Registered Voters 2022,Turnout 2022,projected_Registered_Voters_2027,projected_Turnout_2027,Voters_2027
0,1,MOMBASA,408747,0.6662,580644,0.5896,641913,0.4317,709647.046330,0.52597,709647.046330
1,2,KWALE,174443,0.7200,281102,0.6588,328253,0.5452,383312.932704,0.61424,383312.932704


In [20]:
print(df.projected_Registered_Voters_2027.sum())

24986315.581194714


In [25]:
# 2. Project Voter Turnout for 2027 using Weighted Averages
def project_voter_turnout(turnout_2013, turnout_2017, turnout_2022):
    if not all(isinstance(t, (int, float)) for t in [turnout_2013, turnout_2017, turnout_2022]):
        return None
    if turnout_2013 is None or turnout_2017 is None or turnout_2022 is None:
        return None
    # Calculate weighted average
    weighted_average_turnout = (
        0.20 * turnout_2013 + 0.30 * turnout_2017 + 0.50 * turnout_2022
    )
    return weighted_average_turnout

# Apply the weighted average calculation
df["Turnout_2027"] = df.apply(
    lambda row: project_voter_turnout(
        row["Turnout 2013"], row["Turnout 2017"], row["Turnout 2022"]
    ),
    axis=1,
)

In [26]:
# Print the first few rows of the updated DataFrame to verify the results
print(df.head())

   County Code County Name  Registered Voters 2013  Turnout 2013  \
0            1     MOMBASA                  408747        0.6662   
1            2       KWALE                  174443        0.7200   
2            3      KILIFI                  336132        0.6500   
3            4  TANA RIVER                   79454        0.8100   
4            5        LAMU                   52346        0.8400   

   Registered Voters 2017  Turnout 2017  Registered Voters 2022  Turnout 2022  \
0                  580644        0.5896                  641913        0.4317   
1                  281102        0.6588                  328253        0.5452   
2                  508425        0.6448                  588602        0.4848   
3                  118338        0.7345                  141096        0.6626   
4                   69793        0.7051                   81453        0.6146   

   projected_Registered_Voters_2027  projected_Turnout_2027    Voters_2027  \
0                     7096

In [27]:
#Average Turnout_2027
print(df.Turnout_2027.mean())

0.7312220833333334


In [28]:
# Drop the columns
df = df.drop(columns=["projected_Turnout_2027", "projected_Registered_Voters_2027"])

# Print the first few rows of the updated DataFrame to verify the results
print(df.head())

   County Code County Name  Registered Voters 2013  Turnout 2013  \
0            1     MOMBASA                  408747        0.6662   
1            2       KWALE                  174443        0.7200   
2            3      KILIFI                  336132        0.6500   
3            4  TANA RIVER                   79454        0.8100   
4            5        LAMU                   52346        0.8400   

   Registered Voters 2017  Turnout 2017  Registered Voters 2022  Turnout 2022  \
0                  580644        0.5896                  641913        0.4317   
1                  281102        0.6588                  328253        0.5452   
2                  508425        0.6448                  588602        0.4848   
3                  118338        0.7345                  141096        0.6626   
4                   69793        0.7051                   81453        0.6146   

     Voters_2027  Turnout_2027  
0  709647.046330       0.52597  
1  383312.932704       0.61424  
2  68

In [29]:
# Save the updated DataFrame to a new Excel file
df.to_excel("updated_election_data_2027.xlsx", index=False)
print("\nResults saved to updated_election_data.xlsx")


Results saved to updated_election_data.xlsx
